In [40]:
from snowexsql.db import get_db
from snowexsql.data import ImageData, SiteData, LayerData
from rasterio.plot import show
from sqlalchemy.sql import func
import geoalchemy2.functions as gfunc
from geoalchemy2.types import Raster
from geoalchemy2.shape import to_shape
import geopandas as gpd
from snowexsql.conversions import raster_to_rasterio
from snowexsql.conversions import points_to_geopandas, query_to_geopandas, query_to_pandas
import matplotlib.pyplot as plt
import numpy as np
from datetime import date
import pandas as pd

In [37]:
# Pit Site Identifier of interest
site_name = 'Grand Mesa'

# Connect to the database we made.
db_name = 'snow:hackweek@db.snowexdata.org/snowex'
engine, session = get_db(db_name)

datasets = []
amplitudes = []

# Grab our sites details by site id
#q = session.query(SiteData).filter(SiteData.site_id==site_id)
q = session.query(SiteData.site_id).filter(SiteData.site_name==site_name)
q = q.filter(SiteData.tree_canopy=="No Trees")
q = q.filter(SiteData.date>=date(2020,2,1))
q = q.filter(SiteData.date<=date(2020,3,1))
sites = q.all()
sites = [d[0] for d in sites] 
q_layer = session.query(LayerData.pit_id).filter(LayerData.site_id.in_(sites))
q_layer = q_layer.distinct()
df = query_to_pandas(q_layer, engine)
df.sort_values(by=['pit_id'],ascending=True)

# Grab the rasters, union them and convert them as tiff when done
q_raster = session.query(func.ST_AsTiff(func.ST_Union(ImageData.raster, type_=Raster)))

# Only grab rasters that are the bare earth DEM from USGS
q_raster = q_raster.filter(ImageData.type == 'insar amplitude').filter(ImageData.observers=='UAVSAR team, JPL').filter(ImageData.site_name == "Grand Mesa")

for id in df.pit_id:
    try:
        buffered_pit = get_bufferedPit(id)
        q_raster_new = q_raster.filter(gfunc.ST_Intersects(ImageData.raster, buffered_pit))
        rasters = q_raster_new.all()
        dataset = raster_to_rasterio(session, rasters)[0]
        avg_amplitudes = average_amplitudes(dataset.read(1))
        amplitudes.append((id,avg_amplitudes))
    except TypeError:
        print("I failed")
        continue
print(amplitudes)

[('COGM1C1_20200131', 0.05852198625437666), ('COGM1C1_20200208', 0.05852198625437666), ('COGM1C5_20200212', 0.019295577679073972), ('COGM1N1_20200208', 0.06119450896338208), ('COGM1N20_20200205', 0.04076557850619622), ('COGM1N23_20200211', 0.04076557850619622), ('COGM1N3_20200211', 0.06119450896338208), ('COGM1N5_20200211', 0.03314997386222274), ('COGM1N7_20200211', 0.03314997386222274), ('COGM1S12_20200211', 0.019424712183123916), ('COGM1S13_20200205', 0.019424712183123916), ('COGM1S17_20200208', 0.03164488379256625), ('COGM1S2_20200208', 0.05625945256245757), ('COGM1S8_20200201', 0.019424712183123916), ('COGM2C12_20200212', 0.02365567432643445), ('COGM2C13_20200212', 0.056669779764519035), ('COGM2N13_20200206', 0.04076557850619622), ('COGM2N14_20200211', 0.04076557850619622), ('COGM2N21_20200211', 0.04076557850619622), ('COGM2N48_20200201', 0.05243993803333069), ('COGM2N49_20200210', 0.05243993803333069), ('COGM2N8_20200208', 0.03314997386222274), ('COGM2S10_20200205', 0.019424712183

In [41]:
amp = pd.DataFrame(amplitudes)
amp.to_csv('amplitudes.csv')

In [27]:
buffered_pit= get_bufferedPit('6C34')
print(buffered_pit)
q_raster = q_raster.filter(gfunc.ST_Intersects(ImageData.raster, buffered_pit))
q_raster.all()

01030000202069000001000000210000000000000040c7264100000000ae7c5041e5691a143ec726411a02b68fab7c50417ee7506338c7264172bb6937a97c50414e409f252fc7264178dd2c0ea77c50414e00efb522c72641f61f4229a57c50414114998e13c72641f6174c9ba37c50417124b24402c7264110e39573a27c504133ef4f82efc62641c3b27cbda17c504100000000dcc6264100000080a17c5041cd10b07dc8c62641c3b27cbda17c50418fdb4dbbb5c6264110e39573a27c5041bfeb6671a4c62641f6174c9ba37c5041b2ff104a95c62641f61f4229a57c5041b2bf60da88c6264178dd2c0ea77c50418218af9c7fc6264172bb6937a97c50411b96e5eb79c626411a02b68fab7c50410000000078c6264100000000ae7c50411b96e5eb79c62641e6fd4970b07c50418218af9c7fc626418e4496c8b27c5041b2bf60da88c626418822d3f1b47c5041b2ff104a95c626410ae0bdd6b67c5041bfeb6671a4c626410ae8b364b87c50418fdb4dbbb5c62641f01c6a8cb97c5041cd10b07dc8c626413d4d8342ba7c504100000000dcc6264100000080ba7c504133ef4f82efc626413d4d8342ba7c50417124b24402c72641f01c6a8cb97c50414114998e13c726410ae8b364b87c50414e00efb522c726410ae0bdd6b67c50414e409f252fc726418822d3f1b47c50417ee750

[(None,)]

In [36]:
def get_bufferedPit(pit_id):
    # Distance around the pit to collect data in meters
    buffer_dist = 50

    datasets = []

    # Grab our sites details by site id
    q = session.query(LayerData).filter(LayerData.pit_id==pit_id)
    sites = q.all()

    # Grab the pit location from a single layer
    p = sites[0].geom

    # Convert the point to a pyshapely
    pit = to_shape(p)

    # Convert it to a geopandas dataframe for easy plotting
    df_pit = gpd.GeoSeries(pit)
    q = session.query(gfunc.ST_Buffer(p, buffer_dist))
    buffered_pit = q.all()[0][0]
    return buffered_pit

In [7]:
def average_amplitudes(raster):
    """
    input: 2 dimensional array of list of amplitudes over multiple rasters
    output: average amplitude over entire raster
    """
    #create a variable to average each individual raster
    single_avgs = []
    #iterate over all rasters
    for i in range(len(raster)): 
        #compute and append the mean of the raster 
        single_avgs.append(sum(dataset.read(1)[i])/len(dataset.read(1)[i]))
    
    #compute the average amplitude over all rasters
    avg_amp = sum(single_avgs)/len(single_avgs) 
    
    return avg_amp

In [13]:
print(amplitudes)

[('2C12', 0.02365567432643445)]
